In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
import scipy.sparse as sp
from sksparse.cholmod import cholesky
import scipy.linalg

In [2]:
v, f = igl.read_triangle_mesh('data/cactus.off') # change the input mesh here
labels = np.load('data/cactus.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))

In [3]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [4]:
def get_handle_indices(labels):
    return np.argwhere(labels>0)[:, 0]

In [5]:
def get_free_indices(labels):
    return np.argwhere(labels==0)[:, 0]

In [6]:
def get_factor(v, f, free_indices, handle_indices): # get factor and Afc
    Lw = igl.cotmatrix(v, f)
    M = igl.massmatrix(v, f)
    Minv = sp.diags(1 / M.diagonal())
    A = Lw.dot(Minv).dot(Lw)
    Aff = A[free_indices,:][:,free_indices]
    Afc = A[free_indices,:][:,handle_indices]
    factor = cholesky(Aff)
    return factor, Afc

In [7]:
def smooth_mesh(v, f, handle_indices, free_indices, factor, Afc): # get the smoothed mesh
    handle_vertices = v[handle_indices,:]
    b = -Afc.dot(handle_vertices)
    res = factor(b)
    B = v.copy()
    for i, index in enumerate(free_indices):
        B[index] = res[i]
    return B

In [8]:
def computeBasis(v, f, free_indices, creation=True, farest=None):
    # if creation is True, compute the basis for B, if not, then
    # compute the basis for B' (in this case, farest is provided).
    n = igl.per_vertex_normals(v, f)
    adjList = igl.adjacency_list(f)
    X, Y, Z = np.zeros((free_indices.size, 3)), np.zeros((free_indices.size, 3)), np.zeros((free_indices.size, 3))
    if creation:
        farest = np.zeros(free_indices.size, dtype='int')
    for i, index in enumerate(free_indices):
        X[i] = n[index]
        if creation:
            neighbors = adjList[index]
            dists = [np.linalg.norm(neighbor - v[index]) for neighbor in neighbors]
            farest[i] = neighbors[np.argmax(dists)]
        edgeY = v[farest[i]] - v[index]
        proj = (edgeY.dot(X[i])/X[i].dot(X[i]))*X[i]
        y = edgeY - proj
        y /= np.linalg.norm(y)
        Y[i] = y
        z = np.cross(X[i], y)
        z /= np.linalg.norm(z)
        Z[i] = z
    return X, Y, Z, farest

In [9]:
def displacement(vB, vS, X, Y, Z, free_indices): # compute the displacement
    res = np.zeros((free_indices.size, 3))
    for i, index in enumerate(free_indices):
        x, y, z = X[i], Y[i], Z[i]
        diff = vS[index] - vB[index]
        res[i, 0] = diff.dot(x)
        res[i, 1] = diff.dot(y)
        res[i, 2] = diff.dot(z)
    return res

In [10]:
def position_deformer(target_pos):
    handle_indices = get_handle_indices(labels)
    free_indices = get_free_indices(labels)
    factor, Afc = get_factor(v, f, free_indices, handle_indices)
    B = smooth_mesh(v, f, handle_indices, free_indices, factor, Afc)
    B_prime = smooth_mesh(handle_vertex_positions, f, handle_indices, free_indices, factor, Afc)
    X, Y, Z, farest = computeBasis(B, f, free_indices)
    X_prime, Y_prime, Z_prime, _ = computeBasis(B_prime, f, free_indices, False, farest)
    disp = displacement(B, v, X, Y, Z, free_indices)
    S_prime = B_prime.copy()
    for i, index in enumerate(free_indices):
        S_prime[index] += (disp[i,0]*X_prime[i] + disp[i,1]*Y_prime[i] + disp[i,2]*Z_prime[i])
    return S_prime
pos_f.deformer = position_deformer

## Widget UI
## edit the mesh here
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f, **widgets_wrapper())

/Users/zhongyifan/opt/miniconda3/lib/python3.8/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0736866…

interactive(children=(Dropdown(description='s', options=(1, 2, 3), value=1), FloatSlider(value=0.0, descriptio…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

In [11]:
handle_indices = get_handle_indices(labels)
free_indices = get_free_indices(labels)
factor, Afc = get_factor(v, f, free_indices, handle_indices)

B = smooth_mesh(v, f, handle_indices, free_indices, factor, Afc)

B_prime = smooth_mesh(handle_vertex_positions, f, handle_indices, free_indices, factor, Afc)

X, Y, Z, farest = computeBasis(B, f, free_indices)
X_prime, Y_prime, Z_prime, _ = computeBasis(B_prime, f, free_indices, False, farest)
disp = displacement(B, v, X, Y, Z, free_indices)
S_prime = B_prime.copy()
for i, index in enumerate(free_indices):
    S_prime[index] += (disp[i,0]*X_prime[i] + disp[i,1]*Y_prime[i] + disp[i,2]*Z_prime[i])

q = mp.subplot(v, f, c=labels, shading = {"wireframe": True}, s=[1,4,0])
mp.subplot(B, f, c=labels, shading = {"wireframe": True}, s=[1,4,1], data=q)
mp.subplot(B_prime, f, c=labels, shading = {"wireframe": True}, s=[1,4,2], data=q)
mp.subplot(S_prime, f, c=labels, shading = {"wireframe": True}, s=[1,4,3], data=q)

# From left to right:
# S (input mesh), B (smoothed mesh), B' (deformed/smoothed mesh), S' (final deformation results)